<a href="https://colab.research.google.com/github/SeongwonTak/TIL_swtak/blob/master/DLScratch1Study/DLScratch1_Ch6_Prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 밑바닥부터 시작하는 딥러닝 스터디 준비 6장

상당히 까다로웠던 5장...은 추후에 복습하고 먼저 큰 그림부터 잡기 위해 이번주에 나갈 6장을 준비한다.



## 6.1 매개변수의 갱신





### 확률적 경사하강법, SGD의 복습
- 수식
$$W \leftarrow W - \eta \frac{\partial L}{\partial W} $$


In [ ]:
class SGD:
  def __init__(self, lr = 0.01):
    self.lr = lr

  def update(self, params, grads):
    for key in params.keys():
      params[key] -= self.lr * grads[key]

방향에 따라 기울기가 다를경우, 특히 한 방향의 수렴속도가 지나치게 낮을 경우 SGD가 비효율적이게 된다. 다른 방법이 필요!

### 모멘텀
모멘텀은 물리학에서 사용되는 개념에서 따온 방법으로,
'기울기 방향으로 힘을 받아 물체가 가속된다'는 물리법칙에서 따온 식을 바탕으로 한다.


$$v \leftarrow av - \eta\frac{\partial L}{\partial W} $$
$$W \leftarrow W + v $$

In [ ]:
class Momentum:
  def __init__(self, lr = 0.01, momentum = 0.9):  # a값이 모멘텀
    self.lr = lr
    self.momentum = momentum
    self.v = None
  
  def update(self, params, grads):
    if self.v is None:
      self.v = {}
      for key, val in params.items():
        self.v[key] = np.zeros_like(val)
    
    for key in params.keys():
      self.v[key] = self.momentum * self.v[key] - self.lr*grads[key]
      params[key] += self.v[key]

### AdaGrad

- 신경망 학습에서는 학습율 $\eta$의 값이 너무 작으면 학습 시간이 길어지고, 너무 크면 학습이 제대로 이뤄지지않는다.

- 이를 위해 학습률 감소를 활용 학습을 진행하면서, 점차 학습룰을 줄인다.

$$ h \leftarrow h + \frac{\partial L}{\partial W}\cdot\frac{\partial L}{\partial W}$$
$$ W \leftarrow W - \eta \frac{1}{\sqrt{h}}\frac{\partial L}{\partial W} $$

- 매개변수 학습 과정에서 많이 움직인 변수일수록, 학습률이 낮아짐.

Remark. 다만 과거의 기울기를 계속 제곱해 더해가기에 언젠가는 학습률이 0에 매우 가까워질 것이다. 이를 막기 위해 RMSProp이라는 대안이 존재.
과거의 기울기 반영 비율을 줄이고, 새 기울기 정보를 크게 반영한다.

In [ ]:
class AdaGrad:
  def __init__(self, lr = 0.01):
    self.lr = lr
    self.h = None

  def update(self, params, grads):
    if self.h is None:
      self.h = {}
    for key, val in params, items():
      self.h[key] += grads[key] * grads[key]
      params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key])+ 1e-7)

실제 구현의 예에서는 분모값이 0이 안되도록 막아줘야 한다. 따라서 매우 작은 값을 더한 것이다.

### Adam

AdaGrad + 모멘텀

## 6.2 초기 가중치의 설정

초기 가중치 설정 문제는 중요하다.
만일 가중치 값이 모두 동일할 경우, 오차 역전파법에서는 동일한 값만 갱신되어 문제가 발생하게 된다.

또 다른 문제는 기울기 소실 및 표현력의 제한 문제이다.

ex) NMIST에서 
- 표준편차 1 -> 시그모이드 함수 때문에 기울기가 0으로 가버리기 시작.
- 표준편차 0.01 -> 기울기가 0.5 부근에만 집중 (표현력 제한)

=> Xavier 방법을 사용함

In [ ]:
w = np.random.randn(node_num, node_num) / np.sqrt(node_num)

## 6.3 배치 정규화 (Batch Normalization)

배치 정규화의 이유
- 학습 속도의 개선
- 초깃값 의존도 적음.
- 오버피팅 억제

데이터 분포 정규화 과정에 Batch Norm.을 삽입한다.
미니배치 단위로 평균이 0, 분산이 1이 되도록 정규화한다.

$$ \mu_{B} \leftarrow \frac{1}{m}\sum_{i=1}^{m}x_{i}$$
$$ \sigma_{B}^{2} \leftarrow \frac{1}{m} \sum_{i=1}^{m}
(x_{i}-\mu_{B})^{2}$$
$$ \hat{x_{i}} \leftarrow \frac{x_{i} - \mu_{B}}{\sqrt{\sigma_{B}^{2}+\epsilon}}$$

이제 각 미니배치마다, 확대와 이동을 통해 적합한 값으로 조정하게 된다.

$$ y_{i} \leftarrow \gamma \hat{x_{i}} + \beta $$

## 6.4 오버피팅의 억제

오버피팅이란 훈련 때 사용하지 않는 범용데이터에는 제대로 대응하지 못한, 즉 훈련 데이터에만 적응해버린 결과이다.

### 가중치 감소
큰 가중치에 대해서는 그에 맞는 큰 페널티를 부과한다.
가중치 감소는 모든 가중치 각각의 손실 함수에 $\frac{1}{2}\lambda W^{2}$를 더한다. (여기서 가중치는 $W$, $\lambda$는 정규화 세기 조절)


### 드롭아웃

뉴런을 임의로 삭제하면서 학습하는 방법.
- 훈련 때 은닉층의 뉴런을 무작위로 골라 삭제 -> 삭제 뉴런은 신호전달 X
- 훈련 때 데이터를 흘릴 때마다 삭제할 뉴런 무작위 선택.
- 시험 때는 모든 뉴런에 신호를 전달
- 각 뉴런의 출력에 훈련 대 삭제 안 한 비율을 곱하여 출력

In [ ]:
class Dropout:
  def __init__(self, dropout_ratio = 0.5):
    self.dropout_ratio = dropout_ratio
    self.mask = None

  def forward(self, x, train_flg = True):
    if train_flg:
      self.mask = np.random.rand(*x.shape) > self.dropout_ratio
      return x * self.mask
    else:
      return x * (1.0 - self.dropout_ratio)
  
  def backward(self, dout):
    return dout * self.mask

## 6.5 하이퍼파라미터의 선택
- 적절한 하이퍼파라미터의 값 찾기
-> Validation set을 활용가능하다. (6:2:2. TVT Split)


### 하이퍼파리미터의 최적화

방법은 다음과 같다.
- 0. 하이퍼파리미터 값의 범위 설정
- 1. 설정된 범위에서 하이퍼파라미터 값을 무작위로 추출
- 2. 1단계에서 샘플링한 하이퍼파리미터 값을 사용하여 학습, 검증 데이터로 정확도 평가. (에폭은 작게 설정)
- 3. 1, 2 단계를 특정 횟수 반복하여 정확도 결과를 바탕으로 하이퍼파라미터 범위 좁혀나감.
